# Phân Tích Dữ Liệu

Notebook này tổng hợp các thống kê sâu hơn, bao gồm KPI và phát hiện quan trọng.


In [1]:
import pandas as pd
import numpy as np

DATA_PATH = "/Users/doananh/Documents/Documents - Doan’s MacBook Pro/đồ án DS/project2/data_motobikes.xlsx - Sheet1.csv"

df = pd.read_csv(DATA_PATH)

# Chuẩn hóa giá về triệu VND
def parse_price(series: pd.Series) -> pd.Series:
    cleaned = (
        series.astype(str)
        .str.lower()
        .str.replace("đ", "", regex=False)
        .str.replace("vnđ", "", regex=False)
        .str.replace("vnd", "", regex=False)
        .str.replace("triệu", "", regex=False)
        .str.replace("tr", "", regex=False)
        .str.replace(",", "", regex=False)
        .str.replace(".", "", regex=False)
    )
    return pd.to_numeric(cleaned, errors="coerce") / 1_000_000

df["price_m"] = parse_price(df["Giá"])

top_brands = (
    df.groupby("Thương hiệu")
      .agg(count=("id", "count"), avg_price=("price_m", "mean"), median_price=("price_m", "median"))
      .sort_values("count", ascending=False)
)

top_vehicle_types = (
    df.groupby("Loại xe")
      .agg(count=("id", "count"), avg_price=("price_m", "mean"), median_price=("price_m", "median"))
      .sort_values("count", ascending=False)
)

age_bins = [0, 5, 10, 15, 20, 40]
labels = ["0-5", "5-10", "10-15", "15-20", "20+"]
current_year = 2025

df["vehicle_age"] = current_year - pd.to_numeric(df["Năm đăng ký"], errors="coerce")
df["age_bucket"] = pd.cut(df["vehicle_age"], bins=age_bins, labels=labels, include_lowest=True)

age_summary = (
    df.groupby("age_bucket")
      .agg(count=("id", "count"), avg_price=("price_m", "mean"))
      .dropna()
)

{
    "top_brands": top_brands.head(10),
    "top_vehicle_types": top_vehicle_types,
    "age_summary": age_summary
}


/var/folders/jx/06gzk0815bs1l9gs7v_2k5b00000gn/T/ipykernel_42189/3231973815.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("age_bucket")


{'top_brands':              count   avg_price  median_price
 Thương hiệu                                 
 Honda         4374   63.803822         19.00
 Yamaha        1411   20.964280         13.50
 Piaggio        380   33.103474         25.00
 Suzuki         282   30.363373         17.00
 SYM            256    5.964758          4.50
 Hãng khác      233   13.415472          6.20
 Kawasaki        60   67.297722         41.00
 Kymco           28    9.378571          8.60
 Detech          16    9.593056          9.65
 Ducati          16  152.225000        130.00,
 'top_vehicle_types':               count  avg_price  median_price
 Loại xe                                     
 Tay ga         3600  72.297059          20.0
 Xe số          2189  15.136673          10.0
 Tay côn/Moto   1419  43.355376          22.0,
 'age_summary':             count   avg_price
 age_bucket                   
 0-5          1701  125.367480
 5-10         2048   30.996237
 10-15        1809   23.379392
 15-20     

Kết quả trả về là một dict để tiện in ra từng bảng hoặc chuyển sang Excel/CSV cho báo cáo.
